# Grating Stimulus analysis (Panda extraction / consolidation)
Tobias Rose 2020

# Analysis

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
%matplotlib widget
#%matplotlib qt
import seaborn as sns
sns.set()  # set plot styles
import sys
import os
from pathlib import Path, PureWindowsPath, PurePosixPath
from scipy.signal import convolve
from scipy.stats import zscore, rankdata
from scipy.ndimage import gaussian_filter1d
# from scipy.stats import zscore ### Cave! scipy zscore does not handle NaN!
from ScanImageTiffReader import ScanImageTiffReader
from helpers import parse_SI_header as pSI #own
from tqdm import notebook

import rastermap as rm
import pandas as pd
import pickle

from sklearn.decomposition import NMF

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Function definitions

### Aux loading

In [3]:
def load_auxdata(filename):    
    """ Loads .lvd aux data file - Pieter Goltstein 2020"""
    with open(filename, 'rb') as f:
        # Reset file index
        f.seek(0)
        # Get meta data
        samplingfreq = np.fromfile(f, dtype='>f8', count=1)
        print("Aux sampling frequency = {}Hz".format(samplingfreq))
        n_channels = int(np.fromfile(f, dtype='>f8', count=1))
        print("# channels = {}".format(n_channels))
        timestamp = np.fromfile(f, dtype='>f8', count=1)
        print("timestamp = {}".format(timestamp))
        max_input = np.fromfile(f, dtype='>f8', count=1)
        print("max input = {} V".format(max_input))
        # Read aux data
        auxdata = np.fromfile(f, dtype='>f8')
        n_datapoints = int(auxdata.shape[0]/n_channels)
        print("number of aux datapoints = {}".format(n_datapoints))
        auxdata = np.reshape(auxdata,(n_datapoints,n_channels))
        return auxdata, samplingfreq

### Frame extraction

In [4]:
def get_frame_times(auxdata, Frames_chan):
    """ extracts frame onset times """
    len_aux = len(auxdata)
    pos = np.argwhere(auxdata[0:,Frames_chan] > 0.75 * np.max(auxdata[range(0,len_aux),Frames_chan ])) # work on diff of indices rather than on raw diff to prevent multi-smaple detection in up/ downstrokes
    diffpos = np.argwhere(np.diff(pos[0:,0]) > 1)
    frame_times = pos[diffpos,0]
    
    if  len(frame_times)==0:
        print('get_frame_times WARNING: no frames found')
        frame_times = 1;
        return frame_times
    
    # find onset of first frame
    pos_first = np.argwhere(auxdata[0:,Frames_chan] < 0.5 * np.max(auxdata[range(0,len_aux), Frames_chan]))
    diffpos_first = np.argwhere(np.diff(pos_first[0:,0]) > 1)
    frame_times = np.append(diffpos_first[0], frame_times)
    
    return frame_times

### General stimulus bound extraction

In [5]:
def get_stimIDs(auxdata, stimops):
    """ extracts Stim onset times """
    
    Frames_chan = stimops['Frames_chan']
    Stims_chan = stimops['Stims_chan']
    eye1_chan = stimops['eye1_chan']
    eye2_chan = stimops['eye2_chan']
    level = stimops['level']
    minsample_delta = stimops['minsample_delta']
    extract_stim = stimops['extract_stimID']

    frame_times         = get_frame_times(auxdata,Frames_chan)
    frame_times_level   = frame_times[range(0,len(frame_times),level)]


    StimOn = auxdata[frame_times_level, Stims_chan]>0.8

    # generate cleaned eye binaries
    Eye1On = auxdata[frame_times_level,eye1_chan]*-1+ np.max(auxdata[frame_times_level,eye1_chan])>0.8
    Eye2On = auxdata[frame_times_level,eye2_chan]>0.8
    Eye2On[-1] = 1 
    bino = Eye1On & Eye2On

    Eye1On_only = Eye1On != bino
    Eye2On_only = Eye2On != bino
    Eye1On_only[-1] = False
    Eye2On_only[-1] = False
    bino[0] = False
    bino[-1] = False

    # generate cleaned bino binary
    bino_onsets_temp  = np.argwhere(np.diff(np.multiply(bino, 1)) > 0)
    bino_offsets_temp = np.argwhere(np.diff(np.multiply(bino, 1)) < 0)

    bino_onsets  = bino_onsets_temp[np.argwhere(bino_offsets_temp[0:,0] - bino_onsets_temp[0:,0] > minsample_delta)]
    bino_offsets = bino_offsets_temp[np.argwhere(bino_offsets_temp[0:,0] - bino_onsets_temp[0:,0] > minsample_delta)]                              

    bino_clean = np.full(( len(frame_times_level)), False) 

    for i in range(len(bino_onsets)):
        bino_clean[range(bino_onsets[i,0,0], bino_offsets[i,0,0])] = True

    # extract stim on and offsets
    stim_onsets_temp  = np.argwhere(np.diff(np.multiply(StimOn, 1)) > 0)
    stim_offsets_temp = np.argwhere(np.diff(np.multiply(StimOn, 1)) < 0)

    stim_on  = np.argwhere(np.diff(stim_onsets_temp[0:,0]) > minsample_delta) + 1
    stim_off = np.argwhere(np.diff(stim_offsets_temp[0:,0]) > minsample_delta)

    stim_on  = np.append(0, stim_on)
    stim_off = np.append(stim_off, len(stim_offsets_temp) - 1)

    stim_onsets  = stim_onsets_temp[stim_on]
    stim_offsets = stim_offsets_temp[stim_off]

    ids = { 
            'Contra':   [np.intersect1d(stim_onsets, np.argwhere(Eye1On_only)), np.intersect1d(stim_offsets, np.argwhere(Eye1On_only))],
            'Ipsi': [np.intersect1d(stim_onsets, np.argwhere(Eye2On_only)), np.intersect1d(stim_offsets, np.argwhere(Eye2On_only))],
            'Bino':   [np.intersect1d(stim_onsets, np.argwhere(bino_clean)), np.intersect1d(stim_offsets, np.argwhere(bino_clean))],
            'FrameTimes_level': frame_times_level,
            'FrameTimes':       frame_times,
            }

    ContraStim = []
    IpsiStim = []
    BinoStim = []

    if extract_stim:
        for trial, val in enumerate(ids['Contra'][0]):
            ContraStim.append(np.median(auxdata[frame_times_level, Stims_chan][ids['Contra'][0][trial]:ids['Contra'][1][trial]]))
            IpsiStim.append(np.median(auxdata[frame_times_level, Stims_chan][ids['Ipsi'][0][trial]:ids['Ipsi'][1][trial]]))
            BinoStim.append(np.median(auxdata[frame_times_level, Stims_chan][ids['Bino'][0][trial]:ids['Bino'][1][trial]]))

        C_Stim = np.round(np.array(ContraStim)*10)/10
        I_Stim = np.round(np.array(IpsiStim)*10)/10
        B_Stim = np.round(np.array(BinoStim)*10)/10

        NumStim = len(np.unique(C_Stim))
        TotStim = len(C_Stim)
        TrialNum = int(TotStim / NumStim)
        StimIDs_Contra = rankdata(C_Stim, 'dense')
        StimIDs_Ipsi = rankdata(I_Stim, 'dense')
        StimIDs_Bino = rankdata(B_Stim, 'dense')

        ids.update({ 
                'NumStim':       NumStim,
                'TotStim':       TotStim,
                'TotTrialNum':      TrialNum,
                'StimID_Contra':      StimIDs_Contra,
                'StimID_Ipsi':        StimIDs_Ipsi,
                'StimID_Bino':        StimIDs_Bino,
                'StimDirs_Contra':    StimIDs_Contra / NumStim * 360,
                'StimDirs_Ipsi':      StimIDs_Ipsi / NumStim * 360,
                'StimDirs_Bino':      StimIDs_Bino / NumStim * 360
                    })
    return ids

### on-the-fly panda function definition of zscore that handles NaNs

In [6]:
def z_score(df): return (df-df.mean())/df.std(ddof=0)
# def m_mean(df): return df.mean()

# Batch processing

### User-specific folders

In [7]:
if sys.platform == "darwin":
    ### Mac - command + k in finder; smb://10G.ISI01.neuro.mpg.de/archive_bonhoeffer_group$/David Laubender
    main_root = '/Volumes/David Laubender/Data/imaging data/DL_191024_6/ImagingData/' #location of original data
    adata     = '/Volumes/David Laubender/adata' #location of saved analyzed data
    ftemp     = '/Users/trose/Data/temp' #fast disk (local ssd for s2p binary files) 
    ftiff     = '/Users/trose/Data/s2p_tiff' #fast disk folder for concatenated tiffs (if needed)
    xlpath    = 'LGNexperiments.xlsx' 
elif sys.platform == "win32":
#     main_root = 'I:/David Laubender/Data/imaging data/DL_191106_2/ImagingData' #location of original data
    main_root = 'I:/David Laubender/Data/imaging data/DL_191024_6/ImagingData' #location of original data    
    adata     = 'I:/David Laubender/adata' #location of saved analyzed data
    ftemp     = 'C:/temp/trose/suite2ptemp' #fast disk (local ssd for s2p binary files  
    ftiff     = 'C:/temp/trose/s2p_tiff' #fast disk folder for concatenated tiffs (if needed)
    xlpath    = 'LGNexperiments.xlsx' 

### read excel file into panda dataframe

In [8]:
df = pd.read_excel(xlpath, index_col=0)

### split comments column into Layer and elevation columns and drop mixed column

In [9]:
df[['Layer', 'Elevation']] = df.comments.str.rsplit(',', expand = True)
df = df.drop(['comments'], axis=1)

###  boolean combination of search strings (here: Grating stim only)

In [10]:
# df[(df['experiment type'] == 'euler' ) & (df['Layer'] == 'L1')]
# load_df = df[(df['experiment type'] == 'euler' ) & (df['Layer'] == 'L1') & (df['Elevation']==' positive degrees')]
#load_df = df[(df['experiment type'] == 'euler' ) & (df['Layer'] == 'L4')]

load_df = df[(df['experiment type'] == 'grating' )]
exp=list(map(str,load_df.experiment))
main_root=list(map(str,load_df.folder))
# list(main_root)
# list(exp)
#exp = [exp[0]]
#main_root = [main_root[0]]
#main_root_old = main_root[0]

if sys.platform == "darwin":
    main_root_mac = []
    for idx, val in enumerate(exp):
        main_root_mac.append(str(Path('/Volumes/', *PureWindowsPath(main_root[idx]).parts[1:])))
    main_root = main_root_mac

In [11]:
main_root

['I:\\David Laubender\\Data\\imaging data\\DL_191024_6\\ImagingData\\2019-12-01',
 'I:\\David Laubender\\Data\\imaging data\\DL_191024_6\\ImagingData\\2019-12-01',
 'I:\\David Laubender\\Data\\imaging data\\DL_191024_6\\ImagingData\\2019-12-04',
 'I:\\David Laubender\\Data\\imaging data\\DL_191024_6\\ImagingData\\2019-12-04',
 'I:\\David Laubender\\Data\\imaging data\\DL_191024_6\\ImagingData\\2019-12-08',
 'I:\\David Laubender\\Data\\imaging data\\DL_191024_6\\ImagingData\\2019-12-08',
 'I:\\David Laubender\\Data\\imaging data\\DL_191024_6\\ImagingData\\2019-12-18',
 'I:\\David Laubender\\Data\\imaging data\\DL_191024_6\\ImagingData\\2019-12-18',
 'I:\\David Laubender\\Data\\imaging data\\DL_191024_4\\ImagingData\\2019-11-26',
 'I:\\David Laubender\\Data\\imaging data\\DL_191024_4\\ImagingData\\2019-11-26',
 'I:\\David Laubender\\Data\\imaging data\\DL_191024_4\\ImagingData\\2019-11-29',
 'I:\\David Laubender\\Data\\imaging data\\DL_191024_4\\ImagingData\\2019-11-29',
 'I:\\David Laub

In [12]:
# exp = ['62283', '62284', '62285', '62286', '62287', '62288', '62289', '62290']

In [13]:
# exp = ['62336']

In [14]:
adata_s2ppath = []
adata_tiffpath = []
aux_files = []
single_tiff_file =[]
experimentator = []
mouse = []
date = []
experiment = []

for idx, val in notebook.tqdm(enumerate(exp)):
    s2pdir = list(Path(main_root[idx]).rglob('suite2p_exp'+val+'/')) #recursive
    tifffile = list(Path(os.path.join(*Path(s2pdir[0]).parts[0:-3], 'ImagingData', Path(s2pdir[0]).parts[-2])).glob('exp'+val+'*.tif')) #recursive search over main_root    
    experimentator.append(Path(s2pdir[0]).parts[-7])
    mouse.append(Path(s2pdir[0]).parts[-4])
    date.append(Path(s2pdir[0]).parts[-2])
    experiment.append(val)
    Layer = load_df[load_df['experiment'] == int(val)]['Layer'].str.strip().to_numpy()
    Elevation = load_df[load_df['experiment'] == int(val)]['Elevation'].str.strip().to_numpy()
    try:
        adata_s2ppath.append(os.path.join(s2pdir[0], 'suite2p', 'combined'))
        adata_tiffpath.append(os.path.dirname(s2pdir[0]))
        aux_files.append(*Path(os.path.join(*Path(s2pdir[0]).parts[0:-3], 'data', Path(s2pdir[0]).parts[-2])).glob('exp'+val+'*.lvd'))
        single_tiff_file.append(str(tifffile[0]))
    except:
        print(val + ' not found')

### parse first tiff of exp for imaging specs

In [15]:
with ScanImageTiffReader(single_tiff_file[0]) as reader:
    header = (reader.description(0))
    mov_dim = (reader.shape())
    
level = pSI.parse_SI_header_level(header)
zoom = pSI.parse_SI_header_zoom(header)
framerate = pSI.parse_SI_header_FrameRate(header)
channels = pSI.parse_SI_header_Channels(header)
volumes = pSI.parse_SI_header_Volumes(header)
frames = pSI.parse_SI_header_Frames(header)
frames_per_file = pSI.parse_SI_header_FramesPerFile(header)
#start_time = pSI.parse_SI_header_StartTime(header)

# account for multilevel acq where frames is 1
if frames < volumes:
    frames = volumes

### extract grating stim timebase

In [16]:
stimops = {
    'Frames_chan': 3,
    'Stims_chan': 7,
    'eye1_chan': 16,
    'eye2_chan': 17,
    'level': level, # extract from SI file in the future
    'minsample_delta': 10, 
    'extract_stimID': True
     }

In [17]:
#with open('Grat_ids.pkl', 'rb') as handle:
#    ids = pickle.load(handle)

In [18]:
#auxdata = ids[0]['auxdata']

In [19]:
#level = dataframe['SI_level'].to_numpy()[0]

In [20]:
ids = {}
for idx, val in notebook.tqdm(enumerate(exp)):
    aux_filename = str(aux_files[idx])
    print(aux_filename)
    [auxdata, aux_samplingfreq] = load_auxdata(aux_filename)
    print('extracting stimIDs')
    ids[idx] = get_stimIDs(auxdata, stimops)
    ids[idx]['aux_filename'] = aux_filename
    if idx == 0:
        ids[idx]['auxdata'] = auxdata
    else:
        ids[idx]['auxdata'] = []
    ids[idx]['aux_samplingfreq'] = aux_samplingfreq
    ids[idx]['framerate_auxderived'] = aux_samplingfreq / np.median(np.diff(ids[idx]['FrameTimes']))
print('DONE!')

I:\David Laubender\Data\imaging data\DL_191024_6\data\2019-12-01\exp62283_DL_0013.lvd
Aux sampling frequency = [5000.]Hz
# channels = 19
timestamp = [2.01912011e+13]
max input = [10.] V
number of aux datapoints = 12491520
extracting stimIDs
I:\David Laubender\Data\imaging data\DL_191024_6\data\2019-12-01\exp62286_DL_0022.lvd
Aux sampling frequency = [5000.]Hz
# channels = 19
timestamp = [2.01912012e+13]
max input = [10.] V
number of aux datapoints = 12361984
extracting stimIDs
I:\David Laubender\Data\imaging data\DL_191024_6\data\2019-12-04\exp62289_DL_0023.lvd
Aux sampling frequency = [5000.]Hz
# channels = 19
timestamp = [2.01912041e+13]
max input = [10.] V
number of aux datapoints = 12359936
extracting stimIDs
I:\David Laubender\Data\imaging data\DL_191024_6\data\2019-12-04\exp62292_DL_0033.lvd
Aux sampling frequency = [5000.]Hz
# channels = 19
timestamp = [2.01912041e+13]
max input = [10.] V
number of aux datapoints = 12361984
extracting stimIDs
I:\David Laubender\Data\imaging data

### plot aux_data

#### raw aux plus stimbounds

In [ ]:
# only works if all auxdaa is put into ids dictionary (not with the first slice only)
#fig = plt.figure(figsize=(8,6))
#jump = 0
#for idx, val in enumerate(exp):
#    plt.subplot(len(exp)*3 +1 ,1,idx+jump+1)
#    plotdata = ids[idx]['auxdata'][ids[idx]['FrameTimes_level'],stimops['Stims_chan']]
#    plt.plot(plotdata), plt.ylabel('Stims_chan') 
#    plt.vlines([ids[idx]['Ipsi'][0]],-1,6, 'r')
#    plt.vlines([ids[idx]['Ipsi'][1]],-1,6, 'r')
#    plt.vlines([ids[idx]['Contra'][0]],-1,6, 'b')
#    plt.vlines([ids[idx]['Contra'][1]],-1,6, 'b')
#    plt.vlines([ids[idx]['Bino'][0]],-1,6, 'k')
#    plt.vlines([ids[idx]['Bino'][1]],-1,6, 'k')
#    plt.subplot(len(exp)*3+1 ,1,idx+jump+2)
#    plt.plot(ids[idx]['auxdata'][ids[idx]['FrameTimes_level'],stimops['eye1_chan']]), plt.ylabel('eye1_chan') 
#    plt.subplot(len(exp)*3+1 ,1,idx+jump+3)
#    plt.plot(ids[idx]['auxdata'][ids[idx]['FrameTimes_level'],stimops['eye2_chan']]), plt.ylabel('eye2_chan') 
#    plt.show()
#    jump = jump+2

In [ ]:
#fig = plt.figure(figsize=(8,6))
#plt.subplot(1,1,1)
#plt.plot(ids[idx]['auxdata'][:,stimops['Stims_chan']]), plt.ylabel('Stims_chan') 
#plt.vlines(ids[idx]['FrameTimes_level'][ids[idx]['Ipsi'][0]],-1,6, 'r')
#plt.vlines(ids[idx]['FrameTimes_level'][ids[idx]['Ipsi'][1]],-1,6, 'k')
## plt.vlines(ids['Ipsi'][1],-1,6, 'r')
## plt.vlines(ids['Contra'][0],-1,6, 'b')
## plt.vlines(ids['Contra'][1],-1,6, 'b')
## plt.vlines(ids['Bino'][0],-1,6, 'k')
## plt.vlines(ids['Bino'][1],-1,6, 'k')
## plt.subplot(3,1,2)
## plt.plot(auxdata[ids['FrameTimes_level'],stimops['eye1_chan']]), plt.ylabel('eye1_chan') 
## plt.subplot(3,1,3)
## plt.plot(auxdata[ids['FrameTimes_level'],stimops['eye2_chan']]), plt.ylabel('eye2_chan') 
#plt.show()

### Load ids and auxdata

In [ ]:
# with open('Grat_ids.pkl', 'rb') as handle:
#    ids = pickle.load(handle)

In [ ]:
# auxdata = ids[0]['auxdata']

# make PSTH

### load traces

In [21]:
F = {}
Fneu = {}
spks = {}
stat = {}
ops = {}
iscell = {}

for idx, val in notebook.tqdm(enumerate(exp)):
    F[idx] = np.load(os.path.join(adata_s2ppath[idx],'F.npy'))
    Fneu[idx] = np.load(os.path.join(adata_s2ppath[idx],'Fneu.npy'))
    spks[idx] = np.load(os.path.join(adata_s2ppath[idx],'spks.npy'))
#     stat[idx] = np.load(os.path.join(adata_s2ppath[0],'stat.npy'), allow_pickle=True)
#     ops[idx] = np.load(os.path.join(adata_s2ppath[0],'ops.npy'), allow_pickle=True).item()
    iscell[idx] = np.load(os.path.join(adata_s2ppath[idx],'iscell.npy'))

### drop non-cells

In [22]:
for idx, val in notebook.tqdm(enumerate(exp)):
    old_len = len(F[idx])
    F[idx] = F[idx][iscell[idx][:,0]==1,:]
    Fneu[idx] =Fneu[idx][iscell[idx][:,0]==1,:]
    spks[idx] = spks[idx][iscell[idx][:,0]==1,:]
    print('exp' + val + ' dropped ' + str(old_len - len(F[idx])) + ' non-boutons (out of ' + str(old_len) + ')' )

exp62283 dropped 173 non-boutons (out of 1065)
exp62286 dropped 648 non-boutons (out of 963)
exp62289 dropped 573 non-boutons (out of 1058)
exp62292 dropped 469 non-boutons (out of 725)
exp62305 dropped 453 non-boutons (out of 738)
exp62307 dropped 479 non-boutons (out of 1016)
exp62360 dropped 447 non-boutons (out of 720)
exp62363 dropped 451 non-boutons (out of 679)
exp62258 dropped 266 non-boutons (out of 1207)
exp62261 dropped 741 non-boutons (out of 1235)
exp62266 dropped 478 non-boutons (out of 1180)
exp62269 dropped 263 non-boutons (out of 503)
exp62348 dropped 471 non-boutons (out of 944)
exp62351 dropped 275 non-boutons (out of 398)
exp62335 dropped 489 non-boutons (out of 1012)
exp62338 dropped 281 non-boutons (out of 473)
exp62354 dropped 334 non-boutons (out of 578)
exp62356 dropped 358 non-boutons (out of 910)
exp62399 dropped 426 non-boutons (out of 793)
exp62402 dropped 330 non-boutons (out of 532)
exp62378 dropped 454 non-boutons (out of 684)
exp62381 dropped 273 non-bo

### PANDAS: generate seaborn-compatible, stimulus-chopped long-form pandas and plot (maybe not)

In [23]:
categories = ['Ipsi', 'Contra', 'Bino'] #ids.keys()
stimDir_categories = ['StimDirs_Ipsi', 'StimDirs_Contra', 'StimDirs_Bino'] #ids.keys()
stimID_categories = ['StimID_Ipsi', 'StimID_Contra', 'StimID_Bino'] #ids.keys()


ncells = {}
prestim = {}
poststim = {}
prestim_frames = {}
poststim_frames = {}

for idx, val in enumerate(exp):
    ncells[idx] = F[idx].shape[0]
    prestim[idx]  = 1 / (ids[idx]['framerate_auxderived']  / level) * 15 #seconds before stimulus
    poststim[idx] = 1 / (ids[idx]['framerate_auxderived']  / level) * 15 #seconds after stimulus

    prestim_frames[idx] = np.round(prestim[idx]  * (ids[idx]['framerate_auxderived']  / level))
    poststim_frames[idx] = np.round(prestim[idx]  * (ids[idx]['framerate_auxderived']  / level))

In [24]:
dfs = []
dataframe = []
maxlength = np.empty(3 * len(exp))
jump = 0;

for idx, val in enumerate(exp):
# allocate PSTH array
    maxlength[jump + idx] = np.int(np.ceil(np.diff(ids[idx]["Contra"][:], axis = 0).mean() + prestim_frames[idx] + poststim_frames[idx]))
    maxlength[jump + idx + 1 ] = np.int(np.ceil(np.diff(ids[idx]["Ipsi"][:], axis = 0).mean() + prestim_frames[idx] + poststim_frames[idx]))
    maxlength[jump + idx + 2 ] = np.int(np.ceil(np.diff(ids[idx]["Bino"][:], axis = 0).mean() + prestim_frames[idx] + poststim_frames[idx]))
    jump = jump + 2

maxlength = np.int(np.max(maxlength))
# for cell in tqdm(np.random.randint(0,ncells,5)):

cellglob = 0
data_dict = {}
i = 0


for idx, expval in enumerate(exp):
    #print(cellglob)
    for cell in notebook.tqdm(range(ncells[idx])):
        cat = 0
        for category in categories:            
            for trial, val in enumerate(ids[idx][category][0]):
                aligned_F = np.empty((1,maxlength)).squeeze()
                aligned_F.fill(np.NaN)
                aligned_Fneu = np.empty((1,maxlength)).squeeze()
                aligned_Fneu.fill(np.NaN)
                aligned_spks = np.empty((1,maxlength)).squeeze()
                aligned_spks.fill(np.NaN)
                aligned_stimdata = np.empty((1,maxlength)).squeeze()
                aligned_stimdata.fill(np.NaN)
                slice_range = np.arange(ids[idx][category][0][trial].astype(int)-prestim_frames[idx].astype(int),ids[idx][category][1][trial].astype(int)+poststim_frames[idx].astype(int))
                aligned_stimdata[0:len(slice_range)] = auxdata[ids[idx]['FrameTimes_level'][slice_range],stimops['Stims_chan']]
                aligned_F[0:len(slice_range)] = F[idx][cell,slice_range]
                aligned_Fneu[0:len(slice_range)] = Fneu[idx][cell,slice_range]
                aligned_spks[0:len(slice_range)] = spks[idx][cell,slice_range]

                time = np.arange(0, aligned_F.shape[0] * 1 / (ids[idx]['framerate_auxderived'] / level), 1 / (ids[idx]['framerate_auxderived'] / level) ) - prestim[idx]
                aux_time = np.arange(0, aligned_stimdata.shape[0] * 1 / (ids[idx]['framerate_auxderived'] / level), 1 / (ids[idx]['framerate_auxderived'] / level)  ) - prestim[idx]

                layer = load_df[load_df['experiment'] == int(expval)]['Layer'].to_numpy(),

                data_dict[i] = {
                                        'experimentator': experimentator[0],
                                        'mouse': mouse[0],
                                        'date': date[0],
                                        'experiment': experiment[0],
                                        'cell': cellglob,
                                        'layer': Layer[0],
                                        'elevation': Elevation[0],
                                        'time': time,
                                        'auxtime':aux_time,
                                        'aux_files': aux_files[0],
                                        'aligned_stimdata': aligned_stimdata,
                                        'trial':trial,
                                        'Stimulus_direction': ids[idx][stimDir_categories[cat]][trial],
                                        'Stimulus_ID': ids[idx][stimID_categories[cat]][trial],
                                        'Number_of_stims': ids[idx]['NumStim'],
                                        'Number_of_trials': ids[idx]['TotTrialNum'],
                                        'aligned_F': aligned_F,
                                        'aligned_spks': aligned_spks,
                                        'aligned_Fneu': aligned_Fneu,
                                        'category': category,
                                        'adata_s2ppath': adata_s2ppath[0],
                                        'SI_level': level,
                                        'SI_zoom': zoom,
                                        'SI_framerate': framerate,
                                        'SI_framerate_aux_derived': ids[idx]['framerate_auxderived'][0],
                                        'SI_channels': channels,
                                        'SI_volumes': volumes,
                                        'SI_frames': frames,
                                        'SI_frames_per_file': frames_per_file,
                                        'SI_tiffpath': adata_tiffpath[0],
                                        'SI_single_tiff_file': single_tiff_file[0]
                                        }
                i += 1
            cat += 1
        cellglob += 1

dataframe = pd.DataFrame.from_dict(data_dict, orient = 'index') #MUCH faster than df.append or df.concat! But it does not throw each timepoint in a different row

del data_dict
del F
del Fneu
del spks
del iscell

# EXPLODING to single line - SLOW!
dataframe = dataframe.apply(pd.Series.explode).reset_index(drop=True) #exploding this takes time, though...

# Converting and downcasting object columns to float32
columns = ['time',
           'aligned_F',
           'aligned_spks',
           'aligned_Fneu']
dataframe[columns] = dataframe[columns].apply(pd.to_numeric, errors='coerce', downcast = 'float')

# Convert data column to daaetime
#dataframe['date'] =  pd.to_datetime(dataframe['date'])

# TODO
# EXTRACT ODI SIG ETC HE`RE!
# - include Fneu, dF/F (-Fneu), reconvolved
# - functionalize
# - aggregate over all recordings

In [ ]:
### Save dataframe here
dataframe.to_pickle('layer1_and_4_mac_grat_exp_all_new_dat_sprestim.pkl')

In [ ]:
### Save stim Ids here
with open('Grat_ids_new.pkl', 'wb') as handle:
    pickle.dump(ids, handle, protocol=pickle.HIGHEST_PROTOCOL)

### 'Simple' one-liners to get trial averages over all cells

In [ ]:
allcells_Ipsi_stim1 = dataframe[(dataframe['category'] == 'Bino') & (dataframe['Stimulus_ID'] == 12) & (dataframe['cell'] == 10)].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')

In [ ]:
allcells_Ipsi_stim1 = dataframe[(dataframe['category'] == 'Bino') & (dataframe['Stimulus_ID'] == 12)].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell')

In [ ]:
dataframe.loc[0].Number_of_stims                                                            

In [ ]:
fig = plt.figure(figsize=(12,6))

with sns.axes_style('white'):
    #plt.subplot(2,3,1)
    plt.imshow(allcells_Ipsi_stim1.T, vmax = 2, vmin = 0.5, aspect = 'auto', cmap = 'Reds')c

### concatenate all stimuli

In [ ]:
contra_concat = []
ipsi_concat = []
bino_concat = []

for stimid in notebook.tqdm(range(dataframe.loc[0].Number_of_stims)):
    allcells_Contra_stim = dataframe[(dataframe['category'] == 'Contra') & (dataframe['Stimulus_ID'] == stimid + 1)].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell').to_numpy()
    contra_concat.append(allcells_Contra_stim)
    allcells_Ipsi_stim = dataframe[(dataframe['category'] == 'Ipsi') & (dataframe['Stimulus_ID'] == stimid + 1)].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell').to_numpy()
    ipsi_concat.append(allcells_Ipsi_stim)
    allcells_Bino_stim = dataframe[(dataframe['category'] == 'Bino') & (dataframe['Stimulus_ID'] == stimid + 1)].groupby(['time', 'cell'], as_index=False).aligned_spks.mean().pivot(index = 'time', columns='cell').to_numpy()
    bino_concat.append(allcells_Bino_stim)

contra_concat_np = np.array(contra_concat)
ipsi_concat_np = np.array(ipsi_concat)
bino_concat_np = np.array(bino_concat)

res = contra_concat_np.shape
contra_concat_np = contra_concat_np.reshape(res[0]*res[1],res[2])
ipsi_concat_np = ipsi_concat_np.reshape(res[0]*res[1],res[2])
bino_concat_np = bino_concat_np.reshape(res[0]*res[1],res[2])
all_concat_np = np.concatenate((contra_concat_np, ipsi_concat_np, bino_concat_np), axis = 0)

contra_concat_np[np.isnan(contra_concat_np)] = 0
ipsi_concat_np[np.isnan(ipsi_concat_np)] = 0
bino_concat_np[np.isnan(bino_concat_np)] = 0
all_concat_np[np.isnan(all_concat_np)] = 0


contra_concat_np_zscored = z_score(contra_concat_np)
ipsi_concat_np_zscored = z_score(ipsi_concat_np)
bino_concat_np_zscored= z_score(bino_concat_np)
all_concat_np_zscored= z_score(all_concat_np)

In [ ]:
fig = plt.figure(figsize=(12,6))

with sns.axes_style('white'):
    plt.subplot(1,3,1)
    plt.imshow(contra_concat_np_zscored.T, vmax = 2, vmin = -.5, aspect = 'auto', cmap = 'Blues')
    plt.subplot(1,3,2)
    plt.imshow(ipsi_concat_np_zscored.T, vmax = 2, vmin = -.5, aspect = 'auto', cmap = 'Reds')
    plt.subplot(1,3,3)
    plt.imshow(bino_concat_np_zscored.T, vmax = 2, vmin = -.5, aspect = 'auto', cmap = 'gray_r')

## Rastermap sortings

In [ ]:
%%capture
model_ipsi = rm.mapping.Rastermap(n_components=1).fit(ipsi_concat_np_zscored.T)
model_contra = rm.mapping.Rastermap(n_components=1).fit(contra_concat_np_zscored.T)
model_bino = rm.mapping.Rastermap(n_components=1).fit(bino_concat_np_zscored.T)
model_all = rm.mapping.Rastermap(n_components=1).fit(all_concat_np_zscored.T)

In [ ]:
### sort neurons and smooth across neurons 
smooth = True
timesmooth = True

if smooth:
    isort = np.argsort(model_all.embedding[:,0])   
    Sm_ipsi = gaussian_filter1d(ipsi_concat_np_zscored.T[isort,:], np.minimum(3,int(ipsi_concat_np_zscored.shape[0]*0.005)), axis=0)
    # isort = np.argsort(model_contra.embedding[:,0])
    Sm_contra = gaussian_filter1d(contra_concat_np_zscored.T[isort,:], np.minimum(3,int(contra_concat_np_zscored.shape[0]*0.005)), axis=0)
    # isort = np.argsort(model_bino.embedding[:,0])
    Sm_bino = gaussian_filter1d(bino_concat_np_zscored.T[isort,:], np.minimum(3,int(bino_concat_np_zscored.shape[0]*0.005)), axis=0)
    Sm_all  = gaussian_filter1d(all_concat_np_zscored.T[isort,:], np.minimum(3,int(all_concat_np_zscored.shape[0]*0.005)), axis=0)
#     Sm_all= sp_all.T[isort,:]
else:
    isort = np.argsort(model_bino.embedding[:,0])
    Sm_ipsi = sp_ipsi.T[isort,:]
    # isort = np.argsort(model_contra.embedding[:,0])
    Sm_contra = sp_contra.T[isort,:]
    # isort = np.argsort(model_bino.embedding[:,0])
    Sm_bino = sp_bino.T[isort,:]
    Sm_all  = sp_all.T[isort,:]
    
if timesmooth:
    sigma = 1
# (optional) smooth in time
    Sm_contra = gaussian_filter1d(Sm_contra, sigma, axis=1)
    Sm_ipsi = gaussian_filter1d(Sm_ipsi, sigma, axis=1)
    Sm_bino = gaussian_filter1d(Sm_bino, sigma, axis=1)
    Sm_all = gaussian_filter1d(Sm_all, sigma, axis=1)

In [ ]:
fig = plt.figure(figsize=(12,6))

with sns.axes_style('white'):
    plt.subplot(1,3,1)
    plt.imshow(Sm_contra, vmax = 2, vmin = -0.5, aspect = 'auto', cmap = 'Blues')
    plt.subplot(1,3,2)
    plt.imshow(Sm_ipsi, vmax = 2, vmin = -0.5, aspect = 'auto', cmap = 'Reds')
    plt.subplot(1,3,3)
    plt.imshow(Sm_bino, vmax = 2, vmin = -0.5, aspect = 'auto', cmap = 'gray_r')

In [ ]:
fig = plt.figure(figsize=(12,6))

with sns.axes_style('white'):
    plt.subplot(1,1,1)
    plt.imshow(Sm_all, vmax = 3, vmin = -0.5, aspect = 'auto', cmap = 'gray_r')


### auxdata and timebase 

In [ ]:
slice_range = np.arange(ids[0]['Ipsi'][0][1].astype(int)-prestim_frames[0].astype(int),ids[0]['Ipsi'][1][1].astype(int)+poststim_frames[0].astype(int)+1)
# slice_range = np.arange(ids[category][0][trial].astype(int)-prestim_frames.astype(int),ids[category][1][trial].astype(int)+poststim_frames.astype(int))

aligned_stimdata = auxdata[ids[0]['FrameTimes_level'][slice_range],stimops['Stims_chan']]
# aligned_stimdata_concat = np.tile(auxdata[ids['FrameTimes_level'][slice_range],stimops['Stims_chan']], (0,3))
aux_time = np.arange(0, aligned_stimdata.shape[0] * 1 / (ids[0]['framerate_auxderived'] / level), 1 / (ids[0]['framerate_auxderived'] / level) ) - prestim[0]

aligned_stimdata_full = ids[0]['auxdata'][ids[0]['FrameTimes_level'][slice_range[0]]:ids[0]['FrameTimes_level'][slice_range[-1]],stimops['Stims_chan']]
aligned_stimdata_full_concat = np.tile(aligned_stimdata_full, (1,3)).T

aux_time_full = np.arange(0, aligned_stimdata_full.shape[0] * 1 / ids[0]['aux_samplingfreq'] , 1 / ids[0]['aux_samplingfreq']    ) - prestim[0]
aux_time_full_concat = np.arange(0, aligned_stimdata_full_concat.shape[0] * 1 / ids[0]['aux_samplingfreq'] , 1 / ids[0]['aux_samplingfreq']   ) - prestim[0]

# plt.figure()
# plt.plot(aux_time, aligned_stimdata)
# plt.plot(aux_time_full_concat, aligned_stimdata_full_concat)

### seperately sorted

In [ ]:
### view neuron sorting

# aligned_stimdata = auxdata[ids['FrameTimes_level'][slice_range][0]:ids['FrameTimes_level'][slice_range][-1],stimops['Stims_chan']]
# aux_time = np.arange(0, aligned_stimdata.shape[0] * 1 / aux_samplingfreq, 1 / aux_samplingfreq  ) - prestim

fs = ids[0]['framerate_auxderived'] / level
vmin_glob = -.5
vmax_glob = 2.5
ncells = cellglob 
with sns.axes_style('white'):
    xl = [-prestim[0],Sm_contra.shape[1]/fs-prestim[0]]
    
    fig8 = plt.figure(figsize=(20,16), constrained_layout=True)
    gs1 = fig8.add_gridspec(nrows=8, ncols=3)
    
    fig8.add_subplot(gs1[1:, :-2])      
    fig8_ax=plt.imshow(Sm_contra, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim[0],Sm_contra.shape[1]/fs-prestim[0], 0,Sm_contra.shape[0]], cmap = 'Blues')
    plt.xlabel('time (s)', fontsize=18)    
    
    plt.vlines([prestim_frames[0] / fs-prestim[0], maxlength / fs - poststim_frames[0] / fs -prestim[0] ],0,ncells-1, 'k')
    plt.ylabel('boutons', fontsize=18)     
        
    f8_ax2 = fig8.add_subplot(gs1[1:, 1:-1])
    ax=plt.imshow(Sm_ipsi, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim[0],Sm_ipsi.shape[1]/fs-prestim[0], 0,Sm_ipsi.shape[0]], cmap = 'Reds')
    plt.xlabel('time (s)', fontsize=18)
    
    plt.vlines([prestim_frames[0] / fs-prestim[0], maxlength / fs - poststim_frames[0] / fs -prestim[0] ],0,ncells-1, 'k')
        
    f8_ax3 = fig8.add_subplot(gs1[1:, 2:])
    ax=plt.imshow(Sm_bino, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim[0],Sm_bino.shape[1]/fs-prestim[0], 0,Sm_bino.shape[0]], cmap = 'gray_r')
    plt.xlabel('time (s)', fontsize=18)
    
    plt.vlines([prestim_frames[0] / fs-prestim[0], maxlength / fs - poststim_frames[0] / fs -prestim[0]],0,ncells-1, 'k')

#    plt.colorbar(label='deconvolved F [a.u.]')
   
    f8_ax4 = fig8.add_subplot(gs1[:1, :-2])    
    plt.plot(aux_time_full, aligned_stimdata_full, 'k')
#     plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim],0,4, 'k')
    f8_ax4.set_xlim(*xl)
    f8_ax4.axis('off')
    plt.title('Contra')
    
    f8_ax5 = fig8.add_subplot(gs1[:1, 1:-1])
    plt.plot(aux_time_full, aligned_stimdata_full, 'k')
    f8_ax5.set_xlim(*xl)
    f8_ax5.axis('off')
    plt.title('Ipsi')

    
    f8_ax6 = fig8.add_subplot(gs1[:1, 2:])
    plt.plot(aux_time_full, aligned_stimdata_full, 'k')
    f8_ax6.set_xlim(*xl)
    f8_ax6.axis('off')
    plt.title('Bino')    
    
#     plt.tight_layout()           
    plt.suptitle('z-scored deconvolved Chirp PSTHs - rastermap embedding (sorted on Bino, smoothened over boutons) - exp' + exp[0])
    
    plt.show()

In [ ]:
plt.savefig('separate.eps', format='eps')

### concatenated figure

In [ ]:
### view neuron sorting

# aligned_stimdata = auxdata[ids['FrameTimes_level'][slice_range][0]:ids['FrameTimes_level'][slice_range][-1],stimops['Stims_chan']]
# aux_time = np.arange(0, aligned_stimdata.shape[0] * 1 / aux_samplingfreq, 1 / aux_samplingfreq  ) - prestim

fs = ids[0]['framerate_auxderived'] / level
vmin_glob = -.5
vmax_glob = 2.5

with sns.axes_style('white'):
    xl = [-prestim[0],Sm_all.shape[1]/fs - prestim[0]]
    
    fig9 = plt.figure(figsize=(20,20), constrained_layout=True)
    
    gs1 = fig9.add_gridspec(nrows=12, ncols=3)
    
    fig9.add_subplot(gs1[1:, :])      
    fig9_ax=plt.imshow(Sm_all, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim[0],Sm_all.shape[1]/fs-prestim[0], 0,Sm_all.shape[0]], cmap = 'gray_r')
    plt.xlabel('time (s)', fontsize=18)    
    
#     plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim ],0,ncells-1, 'k')
    plt.ylabel('boutons', fontsize=18)     
    
    plt.vlines([aux_time_full[-1],  +aux_time_full[-1] *2 + poststim[0]],0,ncells-1, 'r')
    plt.vlines([prestim_frames[0] / fs-prestim[0], maxlength / fs - poststim_frames[0] / fs -prestim[0]],0,ncells-1, 'b')
    
    plt.vlines([aux_time_full[-1] + (prestim_frames[0] / fs),  aux_time_full[-1] + (maxlength / fs - poststim_frames[0] / fs ) ],0,ncells-1, 'b')
    plt.vlines([2 * aux_time_full[-1] + (prestim_frames[0] / fs) + prestim[0],  2 * aux_time_full[-1] + (maxlength / fs - poststim_frames[0] / fs ) + prestim[0]],0,ncells-1, 'b')
#     plt.plot(aux_time_full_concat, aligned_stimdata_full_concat, 'k')
    
#     plt.colorbar(label='deconvolved F [zscored]')
#     plt.colorbar(label='deconvolved F [a.u.]')
   
    f9_ax4 = fig9.add_subplot(gs1[:1, :])    
    plt.plot(aux_time_full_concat, aligned_stimdata_full_concat, 'k')
#     plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim],0,4, 'k')
    f9_ax4.set_xlim(*xl)
    f9_ax4.axis('off')
#     plt.title('Contra                                    Ipsi                                    Bino')
    plt.text(aux_time_full[-1] / 2 - 2, 5 , 'Contra', fontsize=16)
    plt.text(aux_time_full[-1] * 1.5-2, 5 , 'Ipsi', fontsize=16)
    plt.text(aux_time_full[-1] * 2.5-2, 5 , 'Bino', fontsize=16)   
    
#     plt.tight_layout()           
    plt.suptitle('z-scored deconvolved F - Chirp PSTHs - rastermap embedding (smoothened over boutons) - exp' + exp[0])
    
    plt.show()

In [ ]:
plt.savefig('all.eps', format='eps')

# NMF on PSTHs

In [ ]:
components = 29
compstr = ['Comp ' + str(idx) for idx in range(components)]

nmf = NMF(n_components=components, solver="mu")

# sp_all[sp_all < 0] = 0
sp_contra[sp_contra < 0] = 0

W = nmf.fit_transform(sp_contra.T)
 
H = nmf.components_

comp_df_H = pd.DataFrame(H.T, columns=compstr)

In [ ]:
Sm_all = nmf.inverse_transform(W)


In [ ]:
rows = components
cols = int(np.ceil(components / rows))

fig, axs = plt.subplots(rows,cols, figsize=(15, rows*2), facecolor='w', edgecolor='k')
# fig.subplots_adjust(hspace = .5, wspace=.001)

k = 0

for ax, d in zip(axs.ravel(), H):
    k += 1
    ax.plot(time, d)
#     ax.plot(d)
    ax.set_title('component' + str(k))
    ax.vlines([prestim_frames[0] / fs-prestim[0], maxlength / fs - poststim_frames[0] / fs -prestim[0]],0,4, 'k')
# plt.figure() 
# for x in 
# sns.heatmap(data = comp_df_H.T)
plt.show()

In [ ]:
with sns.axes_style('white'):
    
    xl = [-prestim[0],Sm_all.shape[1]/fs - prestim[0]]
    
    fig10 = plt.figure(figsize=(20,20), constrained_layout=True)
    
    gs1 = fig10.add_gridspec(nrows=12, ncols=3)
    
    fig10.add_subplot(gs1[1:, :])      
    fig10_ax=plt.imshow(Sm_all, vmin=vmin_glob,vmax=vmax_glob,aspect='auto',extent=[-prestim[0],Sm_all.shape[1]/fs-prestim[0], 0,Sm_all.shape[0]], cmap = 'gray_r')
    plt.xlabel('time (s)', fontsize=18)    
    
#     plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim ],0,ncells-1, 'k')
    plt.ylabel('boutons', fontsize=18)     
    
    plt.vlines([aux_time_full[-1],  +aux_time_full[-1] *2 + poststim[0]],0,ncells-1, 'r')
    plt.vlines([prestim_frames[0] / fs-prestim[0], maxlength / fs - poststim_frames[0] / fs -prestim[0]],0,ncells-1, 'b')
    
    plt.vlines([aux_time_full[-1] + (prestim_frames[0] / fs),  aux_time_full[-1] + (maxlength / fs - poststim_frames[0] / fs ) ],0,ncells-1, 'b')
    plt.vlines([2 * aux_time_full[-1] + (prestim_frames[0] / fs) + prestim[0],  2 * aux_time_full[-1] + (maxlength / fs - poststim_frames[0] / fs ) + prestim[0]],0,ncells-1, 'b')
#     plt.plot(aux_time_full_concat, aligned_stimdata_full_concat, 'k')
    
#     plt.colorbar(label='deconvolved F [zscored]')
#    plt.colorbar(label='deconvolved F [a.u.]')
   
    f10_ax4 = fig10.add_subplot(gs1[:1, :])    
    plt.plot(aux_time_full_concat, aligned_stimdata_full_concat, 'k')
#     plt.vlines([prestim_frames / fs-prestim, maxlength / fs - poststim_frames / fs -prestim],0,4, 'k')
    f10_ax4.set_xlim(*xl)
    f10_ax4.axis('off')
#     plt.title('Contra                                    Ipsi                                    Bino')
    plt.text(aux_time_full[-1] / 2 - 2, 5 , 'Contra', fontsize=16)
    plt.text(aux_time_full[-1] * 1.5-2, 5 , 'Ipsi', fontsize=16)
    plt.text(aux_time_full[-1] * 2.5-2, 5 , 'Bino', fontsize=16)  
    
#     plt.tight_layout()           
    plt.suptitle('z-scored deconvolved F - Chirp PSTHs - rastermap embedding (smoothened over boutons) - exp' + exp[0])
    
    plt.show()

### Rastermap embedding exploration

In [ ]:
# model = rm.mapping.Rastermap(n_components=2, n_X=40,  nPC=400, init='pca')
embedding = model_contra.fit_transform(sp_contra.T)

In [ ]:
np.round(embedding*40)/4

In [ ]:
embedding[isort]


In [ ]:
S = sp_contra.T

NN, NT = S.shape 

# reshape and sum S across neurons to get "components"
nc = 100
NC = int(np.floor(NN / nc))
Sp = np.reshape(S[isort][:nc*NC], (NC, nc, NT))
Sp = Sp.mean(axis=1)

In [ ]:
Sp = np.reshape(S[isort][:nc*NC], (NC, nc, NT))



In [ ]:
S[isort][:nc*NC].shape

In [ ]:
# plot results here
cmap = plt.get_cmap('gist_ncar')
cmap = cmap(np.linspace(0,.97,32))
cmap = cmap[np.random.permutation(32),:]
plt.figure(figsize=(6,6))
# each point is colored based on stimulus identity
istim = np.ones(len(sp_ipsi.T), dtype = 'int')
plt.scatter(out2[:,0],out2[:,1],color=cmap[istim,:],marker='x')
plt.show()

In [ ]:
plt.figure()
plt.plot(melted_aux[[:,0]], melted_aux[[0,1]])

In [ ]:
melted_aux.shape

### Panda convert to long form

In [ ]:
melted = dataframe.melt(id_vars=['category', 'time', 'trial', 'cell'], value_vars=['aligned_F'])

In [ ]:
melted_aux

In [ ]:
### generating trial numpy array from single cell and condition

arr = melted.loc[(melted.loc[:,'trial'] < 8) & (melted.loc[:,'cell'] == 1) & (melted.loc[:,'category'] == 'Bino'),'value'].to_numpy()
arr = arr.reshape(8,maxlength)

### Panda slicing examples

In [ ]:
### plot single cell including individual trials and means - seaborn and other

# generating trial dataframe from single cell and condtion - SINGLE TRIAL
subselected_orig = melted.loc[(melted.loc[:,'trial'] == 1) & (melted.loc[:,'cell'] == 1) & (melted.loc[:,'category'] == 'Contra'),['time', 'value', 'trial']]

# generating trial dataframe from single cell and condtion - ALL TRIALS
subselected_orig = melted.loc[(melted.loc[:,'cell'] == 1) & (melted.loc[:,'category'] == 'Contra'),['time', 'value', 'trial']]

# reshape ('pivot') to index over time and have single trial columns
subselected = subselected_orig.pivot(index = 'time', columns='trial', values='value')

# reshape ('pivot') to index over time and have single trial columns - zscore on the fly
# subselected = subselected_orig.pivot(index = 'time', columns='trial', values='value').transform((lambda x : zscore(x)))

subselected = (subselected - subselected.mean())/subselected.std(ddof=0)

# generate mean from that

subselected_mean = subselected.mean(axis = 1)

# plot seaborn original frame and pivoted mean and trials next to eachother
fig = plt.figure()
plt.subplot(1,3,1)
sns.lineplot(x = 'time', y = 'value', data = subselected, ci = 'sd')
plt.subplot(1,3,2)
plt.plot(subselected, 'k')
plt.plot(subselected_mean, 'r')
plt.subplot(1,3,3)
# sns.heatmap(subselected, cmap = 'cubehelix')
plt.imshow(subselected.T, aspect = 'auto',  cmap = 'cubehelix')

In [ ]:
subselected = subselected_orig.pivot(index = 'time', columns='trial', values='value')
subselected = (subselected - subselected.mean())/subselected.std(ddof=0)

In [ ]:
### plot all trials of a single cell concatenated

fig = plt.figure()
melted.loc[(melted.loc[:,'cell'] == 1) & (melted.loc[:,'category'] == 'Contra'),'value'].plot()

In [ ]:
fig = plt.figure()
sns.set()
with sns.axes_style('white'):
#     plt.imshow(arr, aspect = 'auto',  cmap = 'cubehelix')
    sns.heatmap(subselected_orig, cmap = 'cubehelix')
    plt.show()

In [ ]:
### plot individual cells as line plots and group by condition

g = sns.FacetGrid(melted, col='category', hue='category', row='cell', sharey='row', margin_titles=True)
g.map(sns.lineplot, 'time', 'value', ci='sd')

In [ ]:
# plt.clf()
g = sns.relplot(x="time", y="value",
                 col="category", row = 'cell', hue="category", style="category",
                 kind="line", data=melted, estimator=None, facet_kws={'sharey': True, 'sharex': True})

### plot examples

In [ ]:
# show cells
im = np.zeros((ops['Ly'], ops['Lx']))
ncells = len(stat)

for n in range(0,ncells):
    ypix = stat[n]['ypix'][~stat[n]['overlap']]
    xpix = stat[n]['xpix'][~stat[n]['overlap']]
    im[ypix,xpix] = n+1
fig = plt.figure(figsize=(7,4))
plt.imshow(im)
plt.tight_layout()
plt.show()

In [ ]:
ops['tau'] = .7

In [ ]:
# show mean image
fig = plt.figure(figsize=(8,3))
plt.subplot(1,3,1)
plt.imshow(ops["meanImg"])
plt.subplot(1,3,2)
plt.imshow(ops["meanImgE"])
plt.subplot(1,3,3)
plt.imshow(ops["Vcorr"])
plt.show

In [ ]:
ops['tau'] = 0.5
bouton = 11;

In [ ]:
efilt = np.exp(- np.linspace(0,50,200) / (ops['tau'] * ops['fs']))
#efilt /= efilt.sum()
sout = convolve(spks[bouton,:], efilt)
sout = sout[:spks.shape[1]]

In [ ]:
fig = plt.figure(figsize=(16,4))
plt.plot(F[bouton]-Fneu[bouton] * .7)
plt.show

In [ ]:
plt.plot(sout)
plt.tight_layout()
plt.show
#plt.plot(F[10:])

In [ ]:
plt.plot(spks[11])
plt.show

In [ ]:
plt.figure(figsize=(7,4))
plt.imshow(spks[:100, :5000], vmax = 3, vmin = -0.5, aspect='auto', cmap = 'gray_r')
plt.title('sample of the neural data matrix')
plt.ylabel('boutons') 
plt.xlabel('time [samples]')

In [ ]:
ops

# ToDo

- batch run different deconvolution settings using this code snippet

In [ ]:
# compute deconvolution
from suite2p import dcnv
import numpy as np

tau = 1.0 # timescale of indicator
fs = 30.0 # sampling rate in Hz
neucoeff = 0.7 # neuropil coefficient
# for computing and subtracting baseline
baseline = 'maximin' # take the running max of the running min after smoothing with gaussian
sig_baseline = 10.0 # in bins, standard deviation of gaussian with which to smooth
win_baseline = 60.0 # in seconds, window in which to compute max/min filters

ops = {'tau': tau, 'fs': fs, 'neucoeff': neucoeff,
       'baseline': baseline, 'sig_baseline': sig_baseline, 'win_baseline': win_baseline}

# load traces and subtract neuropil
F = np.load('F.npy')
Fneu = np.load('Fneu.npy')
Fc = F - ops['neucoeff'] * Fneu

# baseline operation
Fc = dcnv.preprocess(Fc, ops)

# get spikes
spks = dcnv.oasis(Fc, ops)